In [68]:
import pandas as pd
import numpy as np

In [69]:
column_names = ['Ticker','Date_YMD','Time','open','high','low','close','volume','extra']

# here add your csv file path
df = pd.read_csv('../nifty-banknifty-data/Nifty-index-data/2021/Yearly Consolidated/NIFTY.csv',names=column_names,header=None)

df

,Ticker,Date_YMD,Time,open,high,low,close,volume,extra
0,NIFTY,2021/01/01,09:16,13997.9,14020.8,13991.3,14013.2,0,0
1,NIFTY,2021/01/01,09:17,14014.8,14018.5,14008.2,14009.0,0,0
2,NIFTY,2021/01/01,09:18,14008.0,14013.1,14005.0,14012.7,0,0
3,NIFTY,2021/01/01,09:19,14013.7,14019.1,14013.7,14016.2,0,0
4,NIFTY,2021/01/01,09:20,14015.5,14017.8,14012.0,14015.5,0,0
...,...,...,...,...,...,...,...,...,...
92253,NIFTY,2021/12/31,15:26,17357.9,17358.5,17353.8,17356.3,0,0
92254,NIFTY,2021/12/31,15:27,17356.1,17359.7,17355.6,17359.3,0,0
92255,NIFTY,2021/12/31,15:28,17358.8,17360.5,17356.2,17359.8,0,0
92256,NIFTY,2021/12/31,15:29,17359.8,17360.9,17351.1,17353.6,0,0


In [70]:
df.drop(columns=['volume','extra'],inplace=True)
df



,Ticker,Date_YMD,Time,open,high,low,close
0,NIFTY,2021/01/01,09:16,13997.9,14020.8,13991.3,14013.2
1,NIFTY,2021/01/01,09:17,14014.8,14018.5,14008.2,14009.0
2,NIFTY,2021/01/01,09:18,14008.0,14013.1,14005.0,14012.7
3,NIFTY,2021/01/01,09:19,14013.7,14019.1,14013.7,14016.2
4,NIFTY,2021/01/01,09:20,14015.5,14017.8,14012.0,14015.5
...,...,...,...,...,...,...,...
92253,NIFTY,2021/12/31,15:26,17357.9,17358.5,17353.8,17356.3
92254,NIFTY,2021/12/31,15:27,17356.1,17359.7,17355.6,17359.3
92255,NIFTY,2021/12/31,15:28,17358.8,17360.5,17356.2,17359.8
92256,NIFTY,2021/12/31,15:29,17359.8,17360.9,17351.1,17353.6


In [71]:
# x = df.where((df['Time'] == '09:16') | (df['Time'] == '15:30'))
# x = x.dropna()
x = df.copy()
x['DateTime'] = pd.to_datetime(x['Date_YMD'] + ' ' + x['Time'])
x.set_index('DateTime', inplace=True)
resampled_df = x.resample('15min').agg({
    'Ticker' : 'first',
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last'
})
# print(resampled_df.shape)
resampled_df.dropna(inplace=True)
temp = resampled_df.loc['2021-08-02 09:15:00':'2021-08-02 15:30:00',:]
print(len(temp))

resampled_df

26


,Ticker,open,high,low,close
DateTime,,,,,
2021-01-01 09:15:00,NIFTY,13997.9,14023.5,13991.3,14018.5
2021-01-01 09:30:00,NIFTY,14018.0,14028.9,14001.2,14024.0
2021-01-01 09:45:00,NIFTY,14024.2,14033.8,14022.5,14032.2
2021-01-01 10:00:00,NIFTY,14032.3,14033.3,14016.3,14025.8
2021-01-01 10:15:00,NIFTY,14025.0,14036.5,14017.6,14035.5
...,...,...,...,...,...
2021-12-31 14:30:00,NIFTY,17353.5,17369.3,17351.4,17368.8
2021-12-31 14:45:00,NIFTY,17369.3,17377.3,17358.1,17362.8
2021-12-31 15:00:00,NIFTY,17363.5,17365.2,17341.8,17356.6


In [72]:
new_df = resampled_df.copy()

#Heiken Ashi Close
new_df['HA_Close'] = (new_df['open'] + new_df['close'] + new_df['high'] + new_df['low'])/4


#Heiken Ashi Open
new_df['HA_Open'] = pd.Series([float('nan') * len(new_df)])
new_df.iloc[0,new_df.columns.get_loc('HA_Open')] = (new_df.iloc[0,new_df.columns.get_loc('open')] + new_df.iloc[0,new_df.columns.get_loc('close')]) / 2

for i in range(1, len(new_df)):
    new_df.iloc[i, new_df.columns.get_loc('HA_Open')] = (new_df.iloc[i - 1]['HA_Open'] + new_df.iloc[i - 1]['HA_Close']) / 2


#Heiken Ashi High
new_df['HA_High'] = new_df[['high', 'HA_Open', 'HA_Close']].max(axis=1)

#Heiken Ashi Low
new_df['HA_Low'] = new_df[['low', 'HA_Open', 'HA_Close']].min(axis=1)

new_df


,Ticker,open,high,low,close,HA_Close,HA_Open,HA_High,HA_Low
DateTime,,,,,,,,,
2021-01-01 09:15:00,NIFTY,13997.9,14023.5,13991.3,14018.5,14007.800,14008.200000,14023.5,13991.3000
2021-01-01 09:30:00,NIFTY,14018.0,14028.9,14001.2,14024.0,14018.025,14008.000000,14028.9,14001.2000
2021-01-01 09:45:00,NIFTY,14024.2,14033.8,14022.5,14032.2,14028.175,14013.012500,14033.8,14013.0125
2021-01-01 10:00:00,NIFTY,14032.3,14033.3,14016.3,14025.8,14026.925,14020.593750,14033.3,14016.3000
2021-01-01 10:15:00,NIFTY,14025.0,14036.5,14017.6,14035.5,14028.650,14023.759375,14036.5,14017.6000
...,...,...,...,...,...,...,...,...,...
2021-12-31 14:30:00,NIFTY,17353.5,17369.3,17351.4,17368.8,17360.750,17363.481548,17369.3,17351.4000
2021-12-31 14:45:00,NIFTY,17369.3,17377.3,17358.1,17362.8,17366.875,17362.115774,17377.3,17358.1000
2021-12-31 15:00:00,NIFTY,17363.5,17365.2,17341.8,17356.6,17356.775,17364.495387,17365.2,17341.8000


In [73]:
new_df.drop(['open','high','low','close'],axis=1,inplace=True)
new_df.columns = ['Ticker','close','open','high','low']
new_df



,Ticker,close,open,high,low
DateTime,,,,,
2021-01-01 09:15:00,NIFTY,14007.800,14008.200000,14023.5,13991.3000
2021-01-01 09:30:00,NIFTY,14018.025,14008.000000,14028.9,14001.2000
2021-01-01 09:45:00,NIFTY,14028.175,14013.012500,14033.8,14013.0125
2021-01-01 10:00:00,NIFTY,14026.925,14020.593750,14033.3,14016.3000
2021-01-01 10:15:00,NIFTY,14028.650,14023.759375,14036.5,14017.6000
...,...,...,...,...,...
2021-12-31 14:30:00,NIFTY,17360.750,17363.481548,17369.3,17351.4000
2021-12-31 14:45:00,NIFTY,17366.875,17362.115774,17377.3,17358.1000
2021-12-31 15:00:00,NIFTY,17356.775,17364.495387,17365.2,17341.8000


In [74]:
new_df['type'] = new_df.apply(lambda row: 'call' if row['close'] > row['open'] else 'put', axis=1)

new_df

,Ticker,close,open,high,low,type
DateTime,,,,,,
2021-01-01 09:15:00,NIFTY,14007.800,14008.200000,14023.5,13991.3000,put
2021-01-01 09:30:00,NIFTY,14018.025,14008.000000,14028.9,14001.2000,call
2021-01-01 09:45:00,NIFTY,14028.175,14013.012500,14033.8,14013.0125,call
2021-01-01 10:00:00,NIFTY,14026.925,14020.593750,14033.3,14016.3000,call
2021-01-01 10:15:00,NIFTY,14028.650,14023.759375,14036.5,14017.6000,call
...,...,...,...,...,...,...
2021-12-31 14:30:00,NIFTY,17360.750,17363.481548,17369.3,17351.4000,put
2021-12-31 14:45:00,NIFTY,17366.875,17362.115774,17377.3,17358.1000,call
2021-12-31 15:00:00,NIFTY,17356.775,17364.495387,17365.2,17341.8000,put


In [75]:

def check_time(index):
    return (index.time() == pd.Timestamp('09:15:00').time()) | (index.time() == pd.Timestamp('15:15:00').time())

mask = new_df.index.map(check_time)

flc = new_df[mask]
flc

,Ticker,close,open,high,low,type
DateTime,,,,,,
2021-01-01 09:15:00,NIFTY,14007.800,14008.200000,14023.500000,13991.300000,put
2021-01-01 15:15:00,NIFTY,14015.425,14035.185230,14035.185230,14001.600000,put
2021-01-04 09:15:00,NIFTY,14099.375,14019.965058,14107.800000,14019.965058,call
2021-01-04 15:15:00,NIFTY,14134.975,14115.239597,14148.000000,14115.239597,call
2021-01-05 09:15:00,NIFTY,14086.925,14134.291149,14134.291149,14048.200000,put
...,...,...,...,...,...,...
2021-12-29 15:15:00,NIFTY,17201.250,17239.863031,17239.863031,17176.700000,put
2021-12-30 09:15:00,NIFTY,17191.950,17203.290758,17217.100000,17146.300000,put
2021-12-30 15:15:00,NIFTY,17215.425,17212.681670,17226.300000,17197.800000,call


In [76]:
def calculate_difference(row):
    previous_row = flc.shift(1).loc[row.name]  
    if pd.isnull(previous_row).any():  
        return np.NAN
    else:
        return row['type'] ==  previous_row['type']

flc['result'] = flc.apply(calculate_difference, axis=1)
flc

/tmp/ipykernel_11498/2591791637.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flc['result'] = flc.apply(calculate_difference, axis=1)


,Ticker,close,open,high,low,type,result
DateTime,,,,,,,
2021-01-01 09:15:00,NIFTY,14007.800,14008.200000,14023.500000,13991.300000,put,NaN
2021-01-01 15:15:00,NIFTY,14015.425,14035.185230,14035.185230,14001.600000,put,True
2021-01-04 09:15:00,NIFTY,14099.375,14019.965058,14107.800000,14019.965058,call,False
2021-01-04 15:15:00,NIFTY,14134.975,14115.239597,14148.000000,14115.239597,call,True
2021-01-05 09:15:00,NIFTY,14086.925,14134.291149,14134.291149,14048.200000,put,False
...,...,...,...,...,...,...,...
2021-12-29 15:15:00,NIFTY,17201.250,17239.863031,17239.863031,17176.700000,put,True
2021-12-30 09:15:00,NIFTY,17191.950,17203.290758,17217.100000,17146.300000,put,True
2021-12-30 15:15:00,NIFTY,17215.425,17212.681670,17226.300000,17197.800000,call,False


In [77]:
flc['gain'] = flc['close'] - flc['open']
flc['gain'] = flc['gain'].abs()
flc

/tmp/ipykernel_11498/3446970349.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flc['gain'] = flc['close'] - flc['open']
/tmp/ipykernel_11498/3446970349.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flc['gain'] = flc['gain'].abs()


,Ticker,close,open,high,low,type,result,gain
DateTime,,,,,,,,
2021-01-01 09:15:00,NIFTY,14007.800,14008.200000,14023.500000,13991.300000,put,NaN,0.400000
2021-01-01 15:15:00,NIFTY,14015.425,14035.185230,14035.185230,14001.600000,put,True,19.760230
2021-01-04 09:15:00,NIFTY,14099.375,14019.965058,14107.800000,14019.965058,call,False,79.409942
2021-01-04 15:15:00,NIFTY,14134.975,14115.239597,14148.000000,14115.239597,call,True,19.735403
2021-01-05 09:15:00,NIFTY,14086.925,14134.291149,14134.291149,14048.200000,put,False,47.366149
...,...,...,...,...,...,...,...,...
2021-12-29 15:15:00,NIFTY,17201.250,17239.863031,17239.863031,17176.700000,put,True,38.613031
2021-12-30 09:15:00,NIFTY,17191.950,17203.290758,17217.100000,17146.300000,put,True,11.340758
2021-12-30 15:15:00,NIFTY,17215.425,17212.681670,17226.300000,17197.800000,call,False,2.743330


In [78]:
flc_t = flc[flc['result'] == True].copy()
flc_f = flc[flc['result'] == False].copy()

flc_f

,Ticker,close,open,high,low,type,result,gain
DateTime,,,,,,,,
2021-01-04 09:15:00,NIFTY,14099.375,14019.965058,14107.800000,14019.965058,call,False,79.409942
2021-01-05 09:15:00,NIFTY,14086.925,14134.291149,14134.291149,14048.200000,put,False,47.366149
2021-01-05 15:15:00,NIFTY,14204.500,14177.093966,14215.600000,14177.093966,call,False,27.406034
2021-01-07 15:15:00,NIFTY,14134.525,14169.849694,14169.849694,14123.100000,put,False,35.324694
2021-01-08 09:15:00,NIFTY,14246.425,14147.743674,14259.200000,14147.743674,call,False,98.681326
...,...,...,...,...,...,...,...,...
2021-12-24 15:15:00,NIFTY,17004.400,17020.186838,17020.186838,16996.400000,put,False,15.786838
2021-12-27 15:15:00,NIFTY,17087.825,17085.927284,17095.300000,17079.300000,call,False,1.897716
2021-12-29 09:15:00,NIFTY,17221.800,17228.501667,17255.800000,17180.900000,put,False,6.701667


In [79]:
mask = flc_t.index.map(lambda x : x.time() == pd.Timestamp('09:15:00').time())
mask2 = flc_f.index.map(lambda x : x.time() == pd.Timestamp('09:15:00').time())
flc_t[mask]
flc_f[mask2]

,Ticker,close,open,high,low,type,result,gain
DateTime,,,,,,,,
2021-01-04 09:15:00,NIFTY,14099.375,14019.965058,14107.800000,14019.965058,call,False,79.409942
2021-01-05 09:15:00,NIFTY,14086.925,14134.291149,14134.291149,14048.200000,put,False,47.366149
2021-01-08 09:15:00,NIFTY,14246.425,14147.743674,14259.200000,14147.743674,call,False,98.681326
2021-01-12 09:15:00,NIFTY,14472.500,14482.996013,14487.700000,14432.800000,put,False,10.496013
2021-01-13 09:15:00,NIFTY,14634.025,14569.780572,14645.800000,14569.780572,call,False,64.244428
...,...,...,...,...,...,...,...,...
2021-12-10 09:15:00,NIFTY,17495.075,17503.187167,17534.300000,17434.800000,put,False,8.112167
2021-12-15 09:15:00,NIFTY,17307.550,17312.870647,17351.200000,17262.200000,put,False,5.320647
2021-12-16 09:15:00,NIFTY,17347.275,17221.817211,17379.300000,17221.817211,call,False,125.457789


In [80]:

flc_t['gain'].sum() + flc_f['gain'].sum()
print(f"Gain : {flc_t['gain'].sum()}")
print(f"Lose : {flc_f['gain'].sum()}")

Gain : 12834.664653941014
Lose : 9609.557665189548
